In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\anmol\AppData\Local\Temp\ipykernel_31444\3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from California"

In [8]:
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Bruno Rocca Barbaresco Rabaja 2000', 'region': 'Barbaresco, Piedmont, Italy', 'variety': 'Red Wine', 'rating': 97.0, 'notes': 'Unbelievable aromas, with rich, ripe plum but also mineral, tobacco and cedar undertones. Full-bodied, with lovely ripe tannins and a unctuous combination of ripe fruit and light toasty oak. Goes on and on. Fabulous. Greatest wine ever from Bruno Rocca. Best after 2007. 1,500 cases made. (JS)'} score: 0.6326937820096987
{'name': 'Altos las Hormigas Gualtallary Malbec 2017', 'region': 'Uco Valley, Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'A solid violet red color reveals a dense, concentrated, compact wine. Notes of spices and minerality make a complex nose. On the palate, a supple entry, dark berries and spices. A very structured and quite wild Malbec, with firm fine-grain tannins, showing its calcareous foundations and columns. The mouth feel is full of thrilling freshness and balance, with a long and persistent finish.'} 

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [10]:
import requests

# Define the local API endpoint and API key
base_url = "http://127.0.0.1:8080/v1"  # Adjust to your server
headers = {"Authorization": "Bearer sk-no-key-required"}

# Define the request payload
payload = {
    "model": "LLaMA_CPP",
    "messages": [
        {"role": "system", "content": "You are a chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from California"},
        {"role": "assistant", "content": "Please provide search results for me."},
    ]
}

# Make the POST request
response = requests.post(f"{base_url}/chat/completions", json=payload, headers=headers)

# Check the response
if response.status_code == 200:
    completion = response.json()
    print(completion['choices'][0]['message']['content'])
else:
    print(f"Error: {response.status_code}, {response.text}")


Here are some amazing Malbec wines from California:

1. Chateau Ste. Michelle Indian Wells 2018 - 92 PTS - WS
2. Caymus Napa Valley 2018 - 93 PTS - WS
3. Silver Oak Alexander Valley 2018 - 94 PTS - WS
4. Faust Napa Valley 2018 - 95 PTS - WS
5. Sterling Vineyards Dry Riesling 2018 - 90 PTS - WS

Please let me know if you would like me to narrow down the options or if you have any specific preferences.</s>
